# టెక్స్ట్ వర్గీకరణ పని

మనం చెప్పినట్లుగా, మనం సులభమైన టెక్స్ట్ వర్గీకరణ పనిపై దృష్టి సారించబోతున్నాము, ఇది **AG_NEWS** డేటాసెట్ ఆధారంగా ఉంటుంది, ఇది వార్తా శీర్షికలను 4 వర్గాలలో ఒకటిగా వర్గీకరించడమే: వరల్డ్, క్రీడలు, వ్యాపారం మరియు సైన్స్/టెక్.

## డేటాసెట్

ఈ డేటాసెట్ [`torchtext`](https://github.com/pytorch/text) మాడ్యూల్‌లో నిర్మించబడింది, కాబట్టి మనం దీన్ని సులభంగా యాక్సెస్ చేసుకోవచ్చు.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

ఇక్కడ, `train_dataset` మరియు `test_dataset` లేబుల్ (క్లాస్ సంఖ్య) మరియు టెక్స్ట్ జంటలను వరుసగా అందించే సేకరణలను కలిగి ఉంటాయి, ఉదాహరణకు:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

కాబట్టి, మన డేటాసెట్ నుండి మొదటి 10 కొత్త శీర్షికలను ముద్రిద్దాం:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

డేటాసెట్లు ఇటరేటర్లు కావడంతో, డేటాను బహుళసార్లు ఉపయోగించాలనుకుంటే దాన్ని లిస్ట్‌గా మార్చుకోవాలి:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## టోకనైజేషన్

ఇప్పుడు మనం టెక్స్ట్‌ను టెన్సర్లుగా ప్రదర్శించగల **సంఖ్యలుగా** మార్చాలి. మనం పద స్థాయి ప్రాతినిధ్యం కావాలంటే, రెండు పనులు చేయాలి:
* టెక్స్ట్‌ను **టోకెన్లుగా** విభజించడానికి **టోకనైజర్** ఉపయోగించాలి
* ఆ టోకెన్ల యొక్క **వాక్యకోశం** తయారు చేయాలి.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

పదసంపదను ఉపయోగించి, మనం సులభంగా మన టోకనైజ్ చేసిన స్ట్రింగ్‌ను సంఖ్యల సెట్‌గా ఎన్‌కోడ్ చేయవచ్చు:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## పదాల సంచయం టెక్స్ట్ ప్రాతినిధ్యం

పదాలు అర్థాన్ని సూచిస్తాయి కాబట్టి, వాక్యంలో వాటి క్రమం పరిగణించకుండా కూడా, ఒక్కొక్క పదాలను చూసి టెక్స్ట్ అర్థాన్ని అర్థం చేసుకోవచ్చు. ఉదాహరణకు, వార్తలను వర్గీకరించేటప్పుడు, *weather*, *snow* వంటి పదాలు *వాతావరణ సూచన*కి సంకేతం ఇవ్వగలవు, మరొకవైపు *stocks*, *dollar* వంటి పదాలు *ఆర్థిక వార్తలు*కి చెందుతాయి.

**పదాల సంచయం** (BoW) వెక్టర్ ప్రాతినిధ్యం అనేది అత్యంత సాధారణంగా ఉపయోగించే సాంప్రదాయ వెక్టర్ ప్రాతినిధ్యం. ప్రతి పదం ఒక వెక్టర్ సూచికకు అనుసంధానించబడుతుంది, వెక్టర్ అంశం ఆ పదం ఒక నిర్దిష్ట డాక్యుమెంట్‌లో ఎన్ని సార్లు వచ్చిందో చూపిస్తుంది.

![పదాల సంచయం వెక్టర్ ప్రాతినిధ్యం మెమరీలో ఎలా ప్రాతినిధ్యం పొందుతుందో చూపించే చిత్రం.](../../../../../translated_images/te/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **గమనిక**: BoWని టెక్స్ట్‌లోని ఒక్కొక్క పదానికి సంబంధించిన ఒక-హాట్-ఎన్‌కోడ్ చేసిన వెక్టర్ల మొత్తం అని కూడా భావించవచ్చు.

క్రింద Scikit Learn పైథాన్ లైబ్రరీ ఉపయోగించి పదాల సంచయ ప్రాతినిధ్యం ఎలా సృష్టించాలో ఒక ఉదాహరణ ఉంది:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

మా AG_NEWS డేటాసెట్ యొక్క వెక్టర్ ప్రాతినిధ్యం నుండి బ్యాగ్-ఆఫ్-వర్డ్స్ వెక్టర్‌ను లెక్కించడానికి, మేము క్రింది ఫంక్షన్‌ను ఉపయోగించవచ్చు:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **గమనిక:** ఇక్కడ మేము గ్లోబల్ `vocab_size` వేరియబుల్‌ను ఉపయోగించి పదసంపద యొక్క డిఫాల్ట్ పరిమాణాన్ని నిర్దేశిస్తున్నాము. సాధారణంగా పదసంపద పరిమాణం చాలా పెద్దదిగా ఉండే కారణంగా, మేము పదసంపద పరిమాణాన్ని అత్యధిక ప్రాముఖ్యత కలిగిన పదాలకు పరిమితం చేయవచ్చు. `vocab_size` విలువను తగ్గించి క్రింద ఇచ్చిన కోడ్‌ను నడిపించి, అది ఖచ్చితత్వంపై ఎలా ప్రభావం చూపుతుందో చూడండి. మీరు కొంత ఖచ్చితత్వం తగ్గుదలని ఆశించవచ్చు, కానీ గణనీయమైనది కాదు, అధిక పనితీరు కోసం.


## BoW క్లాసిఫయర్ శిక్షణ

ఇప్పుడు మనం మన టెక్స్ట్ యొక్క బ్యాగ్-ఆఫ్-వర్డ్స్ ప్రాతినిధ్యం ఎలా నిర్మించాలో నేర్చుకున్నాము, దాని పై క్లాసిఫయర్‌ను శిక్షణ ఇవ్వండి. మొదట, మన శిక్షణ డేటాసెట్‌ను అన్ని స్థానిక వెక్టర్ ప్రాతినిధ్యాలను బ్యాగ్-ఆఫ్-వర్డ్స్ ప్రాతినిధ్యంగా మార్చే విధంగా మార్చాలి. ఇది సాధారణ torch `DataLoader` కు `collate_fn` పారామీటర్‌గా `bowify` ఫంక్షన్‌ను పంపించడం ద్వారా సాధ్యమవుతుంది:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

ఇప్పుడు ఒక సాదా క్లాసిఫయర్ న్యూరల్ నెట్‌వర్క్‌ను నిర్వచిద్దాం, ఇది ఒక లీనియర్ లేయర్‌ను కలిగి ఉంటుంది. ఇన్‌పుట్ వెక్టర్ పరిమాణం `vocab_size` కు సమానం, మరియు అవుట్‌పుట్ పరిమాణం క్లాసుల సంఖ్య (4) కు సరిపోతుంది. మేము క్లాసిఫికేషన్ టాస్క్‌ను పరిష్కరిస్తున్నందున, చివరి యాక్టివేషన్ ఫంక్షన్ `LogSoftmax()` అవుతుంది.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

ఇప్పుడు మనం ప్రామాణిక PyTorch శిక్షణ లూప్‌ను నిర్వచించబోతున్నాము. మన డేటాసెట్ చాలా పెద్దది కాబట్టి, మన బోధనా ప్రయోజనాల కోసం మనం కేవలం ఒక epoch మాత్రమే శిక్షణ ఇస్తాము, మరియు కొన్ని సార్లు ఒక epoch కంటే తక్కువ సమయం కూడా శిక్షణ ఇస్తాము (`epoch_size` పారామీటర్‌ను నిర్దేశించడం ద్వారా శిక్షణను పరిమితం చేయవచ్చు). శిక్షణ సమయంలో సేకరించిన శిక్షణ ఖచ్చితత్వాన్ని కూడా నివేదిస్తాము; నివేదిక ఇచ్చే తరచుదనం `report_freq` పారామీటర్ ద్వారా నిర్దేశించబడుతుంది.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## బైగ్రామ్స్, ట్రైగ్రామ్స్ మరియు ఎన్-గ్రామ్స్

బ్యాగ్ ఆఫ్ వర్డ్స్ పద్ధతికి ఒక పరిమితి ఏమిటంటే, కొన్ని పదాలు బహుళ పదాల వ్యక్తీకరణల భాగంగా ఉంటాయి, ఉదాహరణకు, 'hot dog' అనే పదానికి 'hot' మరియు 'dog' అనే పదాల నుండి పూర్తిగా వేరే అర్థం ఉంటుంది. మనం 'hot' మరియు 'dog' అనే పదాలను ఎప్పుడూ ఒకే వెక్టర్లతో సూచిస్తే, అది మన మోడల్‌ను గందరగోళం చేయవచ్చు.

దీనిని పరిష్కరించడానికి, **ఎన్-గ్రామ్ ప్రాతినిధ్యాలు** డాక్యుమెంట్ వర్గీకరణ పద్ధతుల్లో తరచుగా ఉపయోగిస్తారు, ఇక్కడ ప్రతి పదం, ద్విపదం లేదా త్రిపదం యొక్క సాంద్రత వర్గీకరణ శిక్షణకు ఉపయోగకరమైన లక్షణంగా ఉంటుంది. ఉదాహరణకు, బైగ్రామ్ ప్రాతినిధ్యంలో, మేము అసలు పదాలతో పాటు అన్ని పద జంటలను కూడా పదసంపదలో చేర్చుతాము.

క్రింద స్కైకిట్ లెర్న్ ఉపయోగించి బైగ్రామ్ బ్యాగ్ ఆఫ్ వర్డ్ ప్రాతినిధ్యం ఎలా రూపొందించాలో ఒక ఉదాహరణ ఉంది:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram పద్ధతిలో ప్రధాన లోపం ఏమిటంటే, పదసంపద పరిమాణం చాలా వేగంగా పెరుగుతుంది. వాస్తవంలో, N-gram ప్రాతినిధ్యాన్ని కొంత మితి తగ్గింపు సాంకేతికతలతో కలిపి ఉపయోగించాలి, ఉదాహరణకు *ఎంబెడ్డింగ్స్*, వాటిని మనం తదుపరి యూనిట్‌లో చర్చించబోతున్నాము.

మన **AG News** డేటాసెట్‌లో N-gram ప్రాతినిధ్యాన్ని ఉపయోగించడానికి, ప్రత్యేక ngram పదసంపదను నిర్మించాలి:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


మనం పై కోడ్‌ను ఉపయోగించి క్లాసిఫయర్‌ను శిక్షణ ఇవ్వవచ్చు, కానీ అది చాలా మెమరీ-అసమర్థవంతంగా ఉంటుంది. తదుపరి యూనిట్‌లో, మేము ఎంబెడ్డింగ్స్ ఉపయోగించి బిగ్రామ్ క్లాసిఫయర్‌ను శిక్షణ ఇస్తాము.

> **Note:** మీరు టెక్స్ట్‌లో నిర్దిష్ట సంఖ్య కంటే ఎక్కువ సార్లు కనిపించే ngrams మాత్రమే ఉంచవచ్చు. ఇది అరుదైన బిగ్రామ్స్‌ను తొలగించి, డైమెన్షనాలిటీని గణనీయంగా తగ్గిస్తుంది. దీని కోసం, `min_freq` పారామీటర్‌ను ఎక్కువ విలువకు సెట్ చేసి, వోకాబ్యులరీ పొడవు మార్పును గమనించండి.


## పదం తరచుదనం వ్యతిరేక డాక్యుమెంట్ తరచుదనం TF-IDF

BoW ప్రాతినిధ్యంలో, పదం సంభవాలు పదం స్వయంగా సంబంధం లేకుండా సమానంగా బరువు కలిగి ఉంటాయి. అయితే, తరచుగా వచ్చే పదాలు, ఉదాహరణకు *a*, *in* వంటి పదాలు, ప్రత్యేక పదాల కంటే వర్గీకరణకు తక్కువ ప్రాముఖ్యత కలిగి ఉంటాయి. వాస్తవానికి, చాలా NLP పనుల్లో కొన్ని పదాలు ఇతరాల కంటే ఎక్కువ సంబంధితంగా ఉంటాయి.

**TF-IDF** అంటే **term frequency–inverse document frequency**. ఇది బాగ్ ఆఫ్ వర్డ్స్ యొక్క ఒక రూపం, ఇందులో డాక్యుమెంట్‌లో పదం కనిపించడం కోసం 0/1 బైనరీ విలువ బదులు, పదం సంభవాల తరచుదనంతో సంబంధం ఉన్న ఫ్లోటింగ్-పాయింట్ విలువ ఉపయోగిస్తారు.

మరింత సాందర్భికంగా, డాక్యుమెంట్ $j$ లో పదం $i$ యొక్క బరువు $w_{ij}$ ఈ విధంగా నిర్వచించబడుతుంది:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ఇక్కడ
* $tf_{ij}$ అనేది $j$ లో $i$ పదం సంభవాల సంఖ్య, అంటే మునుపటి BoW విలువ
* $N$ అనేది సేకరణలో డాక్యుమెంట్ల సంఖ్య
* $df_i$ అనేది మొత్తం సేకరణలో $i$ పదం ఉన్న డాక్యుమెంట్ల సంఖ్య

TF-IDF విలువ $w_{ij}$ ఒక పదం డాక్యుమెంట్‌లో ఎంతసార్లు కనిపిస్తుందో అనుపాతంగా పెరుగుతుంది మరియు ఆ పదం ఉన్న డాక్యుమెంట్ల సంఖ్యతో తగ్గుతుంది, ఇది కొన్ని పదాలు ఇతరాల కంటే ఎక్కువగా కనిపించే వాస్తవాన్ని సరిచేస్తుంది. ఉదాహరణకు, ఒక పదం సేకరణలో *ప్రతి* డాక్యుమెంట్‌లో కనిపిస్తే, $df_i=N$, మరియు $w_{ij}=0$, అప్పుడు ఆ పదాలను పూర్తిగా పరిగణించరు.

మీరు సులభంగా Scikit Learn ఉపయోగించి టెక్స్ట్ యొక్క TF-IDF వెక్టరైజేషన్ సృష్టించవచ్చు:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## ముగింపు

TF-IDF ప్రాతినిధ్యాలు వేర్వేరు పదాలకు ఫ్రీక్వెన్సీ బరువు ఇస్తున్నప్పటికీ, అవి అర్థం లేదా క్రమాన్ని ప్రతిబింబించలేవు. 1935లో ప్రసిద్ధ భాషావేత్త జె. ఆర్. ఫర్త్ చెప్పినట్లుగా, "ఒక పదం యొక్క పూర్తి అర్థం ఎప్పుడూ సందర్భానుసారంగా ఉంటుంది, మరియు సందర్భం లేకుండా అర్థం అధ్యయనం సీరియస్‌గా తీసుకోలేము." ఈ కోర్సులో తర్వాత మనం భాషా మోడలింగ్ ఉపయోగించి టెక్స్ట్ నుండి సందర్భ సమాచారాన్ని ఎలా పొందాలో నేర్చుకుంటాము.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ డాక్యుమెంట్‌ను AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల డాక్యుమెంట్ దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
